In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
from scipy.stats import sem
import matplotlib.ticker as mtick
from scipy import interpolate
from scipy.interpolate import griddata
from matplotlib import ticker, cm
from matplotlib.patches import Rectangle
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from scipy.stats import norm
import matplotlib.mlab as mlab
from stl import mesh
import math
import random
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

## Plotting Annular Averages from Zemax - Linear and Log scale

In [ ]:
'''
Whena loading in Zemax files...open the txt file, ctrl+f "Tri" and delete everything after (including "tri"). ctrl+s save then open here
Change file names for local machine
'''
array = np.loadtxt(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\5_7_2021\ScanIII_10Mrays_0to2in10rays_150x150mm.txt', skiprows=28, usecols=range(1,722))

print(array.shape)
#make an array of polar angle values called theta
#these are the 'bin boundaries', not the bin midpoints.
#notice there is one more element in the array than the number of annuli (because they're boundaries)
theta = np.arange(0,90,.125) + .0625
theta = np.append(theta, 90)

annavg = np.zeros(721)

for i in range(721):
    annavg[i] = np.mean(array[:,i])
      
print(len(theta))    
print(len(annavg))

plt.scatter(theta, annavg)
plt.title("Annular Power/Str as a Function of Angle\n Scanned Surface scanned 0-2°")
plt.ylabel("Power/Str per Annulus (W/str)")
plt.xlabel("Angle from Normal (degrees)")

plt.figure()
#plt.scatter(theta, annavg)
plt.semilogy(theta, annavg, marker='o')
plt.title("Annular Power/Str as a Function of Angle")
plt.ylabel("Power/Str per Annulus (W/str)")
plt.xlabel("Angle from Normal (degrees)")

## Making Own Polar Plots from Zemax Data

In [ ]:
'''
Download the pp and tt data included on Github
Change all file names for local machine
'''

N = 250 #plots out to the Nth radial pixel

array = np.loadtxt(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\2_12_2021\30degcutNoDeadSpace_50x50rays10Mrays.txt', skiprows=28, usecols=range(1,722))
array = array[:,:N]

test=str()
pp = np.load(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\2_19_2021\pp.npy')
pp = pp[:,:N]

tt = np.load(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\2_19_2021\tt.npy')
tt = tt[:,:N]

print(np.shape(array))

def polar_plot(z, name):
    fig = plt.figure()
    plt.subplot(projection="polar")
    plt.title(name, y=1.08)
    #plt.pcolormesh(-pp, tt * (180/np.pi), z, vmin = np.amin(array)-0.1, vmax=np.amax(array)) #vmax=5
    #plt.pcolormesh(-pp, tt * (180/np.pi), z, vmin = 0.00001, vmax=5) #vmax=5
    plt.pcolormesh(-pp, tt * (180/np.pi), z, norm=colors.LogNorm(vmin = 0.00001, vmax=5))
    plt.colorbar(label='Watts / Sr', pad=.15)
    plt.plot()
    plt.grid(alpha=.5, label='angle')
    ang = tt * (180/np.pi)
    plt.title("Polar Plot Out to\n"+str(np.amax(ang))+"°\n Max Pix Val is "+str(np.amax(array)))

polar_plot(array, test)
print(np.amax(array))    
    
    

## Generating the 2D Noise Surfaces

In [ ]:
### INITIALIZE ###
L = 153 #length of side of sample - mm - 81mm
H = 153 # mm - should be something like 42mm
Nl = L * 2 #would L*2 be the Nyquist?
Nh = H * 2
dL = L/Nl #space between sample points
dH = H/Nh #these are equal - as they should be (?)
#print(dL,dH)
f = 100 #Hz or cycles/mm 75 works well for visualization
wc = f / (2*np.pi) #rad/sec or rad/mm
amp = 1 #amplitude +/- of noise
desired_featuresize = 5 #mm across
safety_mult = 2.8 #Extra multiplier term to even further increase % of 45deg facets


x = np.arange(0,L,dL)
y = np.arange(0,H,dH)
X,Y = np.meshgrid(x,y)

### ESTABLISH NOISE MAP ###
wn2d = np.random.uniform(-amp,amp,(Nh,Nl))
plt.figure()
plt.contourf(X,Y,wn2d)
plt.colorbar()
plt.title("Noise Map")
plt.xlabel("Length (mm)")
plt.ylabel("Height (mm)")

### MAKE A PLANE WAVE ###

#wn2d = amp * np.sin(0.15 * Y) #########################
plt.figure()
plt.contourf(X,Y,wn2d)
plt.colorbar()
plt.title("Noise Map")
plt.xlabel("Length (mm)")
plt.ylabel("Height (mm)")

### TAKE 2D FFT ###
wn2d_fft = np.fft.fft2(wn2d)
plt.figure()
plt.contourf(X,Y,wn2d_fft)
plt.colorbar()
plt.title("FFT of Noise")
plt.xlabel("NOT frequency")
plt.ylabel("NOT frequency")

### SHIFT THE FFT TO 0 ###
wn2d_shift = np.fft.fftshift(wn2d_fft)
plt.figure()
plt.contourf(X,Y,wn2d_shift)
plt.colorbar()
plt.title("Shifted FFT of Noise")
plt.xlabel("NOT frequency")
plt.ylabel("NOT frequency")

### CONVERT TO FERQUENCY AXIS DOMAIN ### I SUSPECT HERE IS WHERE THE ASYMMETRY HAPPENS
#freq bins - THESE NEED CHECKED
print(dL)
#freqs = np.fft.fftfreq((len(x),len(y)))
freqx = np.fft.fftshift(np.fft.fftfreq(x.shape[0],dL))
freqy = np.fft.fftshift(np.fft.fftfreq(y.shape[0],dH))
fX,fY = np.meshgrid(freqx, freqy)
#print(freqx)
#print(freqy)
plt.figure()
plt.contourf(fX,fY,wn2d_shift)
plt.colorbar()
plt.title("Shifted FFT of Noise")
plt.xlabel("Angular Frequency")
plt.ylabel("Angular Frequency")

### MAKE THE LP FILTER ###
#make lp filter
rows, cols = wn2d.shape
crow, ccol = int(rows/2), int(cols/2)
#lpf = np.zeros((rows, cols, 2))
lpf = np.zeros((rows, cols))
center = [crow, ccol]
xlpf, ylpf = np.ogrid[:rows, :cols]
lpf_area = (xlpf - center[0])**2 + (ylpf - center[1])**2 <= wc**2
lpf[lpf_area] = 1
#lpf = np.ones((Nl,Nh)) ############################ THIS CAN BE UNCOMMENTED TO STOP FILTER
#print(lpf)
plt.figure()
plt.contourf(freqx,freqy,lpf) #scaled from angular freq to Hz
plt.plot([0.111, 0.111], [-1, 1], '-k')
plt.colorbar()
plt.title("LP Filter")
plt.xlabel("Frequency (cycles/mm)")
plt.ylabel("Frequency (cycles/mm)")

### APPLY THE FILTER TO THE SHIFTED FFT ###
fftwn2d_lp = lpf*wn2d_shift * 10 ###THIS MULTIPLIER HERE WILL CHANGE THE DERIV CALC
plt.figure()
plt.contourf(2*np.pi*fX, 2*np.pi*fY, fftwn2d_lp)
plt.colorbar()
plt.title("Filter Applied to Shifted Noise FFT")
plt.xlabel("Frequency (cycles/mm)")
plt.ylabel("Frequency (cycles/mm)")

### UNSHIFT THE FILTERED FFT ###
fftwn2d_lp_unshift = np.fft.ifftshift(fftwn2d_lp)
plt.figure()
plt.contourf(2*np.pi*fX, 2*np.pi*fY, fftwn2d_lp_unshift)
plt.colorbar()
plt.title("Unshifted FFT of Filtered Noise Map")
plt.xlabel("Frequency (cycles/mm)")
plt.ylabel("Frequency (cycles/mm)")

### TAKE THE INVERSE FFT TO GET A FILTERED SIGNAL BACK ###
wn2d_lp = np.fft.ifft2(fftwn2d_lp_unshift)
plt.figure()
plt.contourf(X,Y,wn2d_lp)
plt.colorbar()
plt.title("LP Noise Map")
plt.xlabel("Length (mm)")
plt.ylabel("Height (mm)")

### AMPLIFYING FEATURE SIZES TO INCREASE DERIVATIVES ###
#for significant % of 45deg derivatives
scale = desired_featuresize / 2
map_max = np.amax(np.abs(wn2d_lp))
multiplier = scale / map_max
wn2d_lp = wn2d_lp * multiplier * safety_mult
plt.figure()
plt.contourf(X,Y,wn2d_lp)
plt.colorbar()
plt.title("LP Noise Map with Amp Multiplier")
plt.xlabel("Length (mm)")
plt.ylabel("Height (mm)")



print(wn2d_lp)
'''
Change these below and uncomment to save appropriate surface
'''
#np.save('Random_Maps/wn2d_L42_W42_fc75_A1.npy', wn2d_lp)

#wn2d_lp_real = wn2d_lp.real
#wn2d_lp_real = ['{:f}'.format(item) for item in wn2d_lp_real]

#np.savetxt('lptest.txt', wn2d_lp.real, delimiter=',', fmt='%f')


## RMS

In [ ]:
'''
Change the file naming for your local machine below
'''

s = np.loadtxt(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\RandMap6x6w45deg.csv', delimiter=',')

#xsine = np.linspace(0,4*2*np.pi, 150)
#s = np.sin(xsine) ###works fine for a sine func
#plt.plot(xsine, s)
plt.figure()
s1d = s.flatten()
mu = np.mean(s1d)
print("mu = ",mu)
xi2 = (s1d - mu)**2
print(s1d[0:3])
print(xi2[0:3])

xsum = np.sum(xi2)
RMSE = np.sqrt(xsum / (len(xi2)))
print("RMSE = ",RMSE)

plt.imshow(s)
plt.colorbar(label="mm")
print("Max val = ",np.amax(s1d),"Min val = ", np.amin(s1d))
plt.title("White Noise Surface")
plt.xlabel("mm")
plt.ylabel("mm")

plt.figure()
n, bins, patches = plt.hist(s1d, bins=50)
plt.axvline(mu-RMSE, color='r')
plt.axvline(mu+RMSE, color='r')

# def gaussfit(x, sigma):
#     A = 1 / (sigma*np.sqrt(2*np.pi))
#     return A * np.exp(-((x-mu)**2)/(2*sigma**2))
xfit = np.linspace(np.amin(s1d)-1, np.amax(s1d)+1, 1000)

# fit, cov = curve_fit(gaussfit, xfit, gaus)
(mu2, std) = norm.fit(s1d)

y = mlab.normpdf( bins, mu2, std)
l = plt.plot(bins, y, 'r--', linewidth=2)
plt.title("Model Surface RMS")
plt.xlabel("mm")
plt.ylabel("#")

## Ploting Derivative of an STL surface (Model STL surface or 3D scanned STL surface)

In [ ]:
figure = plt.figure()
axes = mplot3d.Axes3D(figure)

your_mesh = mesh.Mesh.from_file(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\PrintedSurfDataFiles\ScannedSurfScaled.stl')
#your_mesh = mesh.Mesh.from_file(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\PrintedSurfDataFiles\RandSurfScanIII\.stl')
data = your_mesh.vectors

#data = data[your_mesh.z>0] #uncomment this if there is error message and STL has z=0 components

xpre = data[:,:,0] #can be flattened or unflattened for the axis.scatter to work
ypre = data[:,:,1]
zpre = data[:,:,2]

ntriangles = len(data[:,0,0])
#print(data)
#print(data[1,2,0])
#print(data[1,2,1])

dtot = []

for i in range(ntriangles):
    o = (data[i,0,0], data[i,0,1], data[i,0,2]) #origin point for each triangle
    x1 = data[i,1,0] #all of this for one point of the triangle
    y1 = data[i,1,1]
    z1 = data[i,1,2]
    
    x2 = data[i,2,0] #all this for the second point on the triangle
    y2 = data[i,2,1]
    z2 = data[i,2,2]
    
    dx1 = (x1-o[0]) / 2 #this is bc its plotted with units of mm/2
    dx2 = (x2-o[0]) / 2
    dy1 = (y1-o[1]) / 2
    dy2 = (y2-o[1]) / 2
    dz1 = z1-o[2]
    dz2 = z2-o[2]
    
    dydx1 = np.sqrt(dx1**2 + dy1**2)
    dydx2 = np.sqrt(dx2**2 + dy2**2)+0.0000001 #this is to avoid -inf
    
    dzdydx1 = dz1/dydx1
    dzdydx2 = dz2/dydx2
    
    #derivtemp =np.array(dzdydx1, dzdydx2)
    #dtot.append(derivtemp)
    dtot.append(dzdydx1)
    dtot.append(dzdydx2)
    
#print(np.array(dtot)) 


###left off with plotting
plt.figure()
plt.hist(dtot, bins=50)
plt.figure()
plt.hist(np.rad2deg(np.arctan(np.abs(dtot))), bins=50)
#plt.title("Discrete Derivs Using STL Triangles\n to Ensure Adjacent Points\n Model Surface")
plt.title("Discrete Derivatives of Scanned STL Surface")
plt.xlabel("Facet Angle (°)")
plt.ylabel("# of Facets ")
    
print(np.amin(np.abs(dtot)))
    
    

x = xpre[zpre!=0]
y = ypre[zpre!=0]
z = zpre[zpre!=0]

plt.ylim(0,40000)
plt.axvline(45, color='r', label="Marker at 45°")
plt.xlim(0,80)
plt.ylim(0,9000)
plt.legend()
'''
Luckily these surfaces have no z vectors = 0
'''
degs = np.rad2deg(np.arctan(np.abs(dtot)))
over45 = (degs>=45).sum() #len(np.where(degs>=45))
total = (degs>=1).sum() #len(np.where(degs>=0))

print("Percent of surface over 45° = ",round(over45/total * 100), 3,"%")

## RMS of STL file (3D scanned surface for ex.)

In [ ]:
#your_mesh = mesh.Mesh.from_file(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\PrintedSurfDataFiles\RandSurf45degIII.stl')
your_mesh = mesh.Mesh.from_file(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\PrintedSurfDataFiles\RandSurfScanIII\SurfScanIIISTLII.stl')

data = your_mesh.vectors

xpre = data[:,:,0]#.flatten()
ypre = data[:,:,1]#.flatten()
zpre = data[:,:,2]#.flatten()
x = xpre[zpre>0]
y = ypre[zpre>0]

your_mesh.z[zpre>0] = (your_mesh.z[zpre>0] - np.mean(your_mesh.z[zpre>0])) * 2 #remove *2

s = your_mesh.z[zpre!=0] #np.loadtxt(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\RandMap6x6w45deg.txt')
# s = np.loadtxt(r'C:\Users\Jared\Documents\Case\PHYS352_CodeStuff\RandMap6x6w45degII.dat')
#xsine = np.linspace(0,4*2*np.pi, 150)
#s = np.sin(xsine) ###works fine for a sine func
#plt.plot(xsine, s)
plt.figure()
s1d = s#.flatten()
mu = np.mean(s1d)
print("mu = ",mu)
xi2 = (s1d - mu)**2
print(s1d[0:3])
print(xi2[0:3])

xsum = np.sum(xi2)
RMSE = np.sqrt(xsum / (len(xi2)))
print("RMSE = ",RMSE)

#plt.imshow(s)
#plt.colorbar()
print("Max val = ",np.amax(s1d),"Min val = ", np.amin(s1d))


plt.figure()
n, bins, patches = plt.hist(s1d, bins=50)
plt.axvline(RMSE-mu, color='r')
plt.axvline(-RMSE-mu, color='r')
plt.title("RMS of Scanned STL")
plt.xlabel("mm deviation")
plt.ylabel("#")
'''
Dont trus this because there is a slant in the data
'''